Prerequisites for installing the bash-kernel

In [ ]:
!pip install bash_kernel
!python -m bash_kernel.install

Create an local cluster for our experiments

In [2]:
%%sh

kubectl get nodes
kubectl get pods -A

NAME                 STATUS   ROLES           AGE   VERSION
kind-control-plane   Ready    control-plane   10m   v1.26.3
NAMESPACE            NAME                                         READY   STATUS    RESTARTS   AGE
kube-system          coredns-787d4945fb-hdjdt                     1/1     Running   0          10m
kube-system          coredns-787d4945fb-ztnmd                     1/1     Running   0          10m
kube-system          etcd-kind-control-plane                      1/1     Running   0          10m
kube-system          kindnet-z96vh                                1/1     Running   0          10m
kube-system          kube-apiserver-kind-control-plane            1/1     Running   0          10m
kube-system          kube-controller-manager-kind-control-plane   1/1     Running   0          10m
kube-system          kube-proxy-zqxfq                             1/1     Running   0          10m
kube-system          kube-scheduler-kind-control-plane            1/1     Running   0   

Check your local age-key file and encode it with base64


In [28]:
%%sh 

sed -n -e '/^AGE-SECRET-KEY/p' ~/.sops/keys/sample-env.txt | base64

QUdFLVNFQ1JFVC1LRVktMUU4S1ZIRDA2VTVENTRMMkgzTDlINlJGOVJFRDNBTUNOUVpSTVM3Szk4
TlNKNlRTWFpEMFNDN1dMNUoK


We need to bootstrap our cluster with an inital secret. So we upload a secret to our cluster with a base64 encoded age key

In [31]:
%%sh

cat <<EOF | kubectl apply -f -
apiVersion: v1
data:
  key: QUdFLVNFQ1JFVC1LRVktMUU4S1ZIRDA2VTVENTRMMkgzTDlINlJGOVJFRDNBTUNOUVpSTVM3Szk4TlNKNlRTWFpEMFNDN1dMNUo=
kind: Secret
metadata:
  name: sops-age-key-file
  namespace: sops
type: Opaque
EOF

kubectl get secrets -n sops

secret/sops-age-key-file unchanged
NAME                         TYPE                 DATA   AGE
sh.helm.release.v1.sops.v1   helm.sh/release.v1   1      7m5s
sops-age-key-file            Opaque               1      15s


Configure our age-secret as values file for the sops-secrets-operator

In [35]:
%%sh

cat >./kubernetes/sops-helm-values.yaml<< EOF
extraEnv:
  - name: SOPS_AGE_KEY_FILE
    value: /etc/sops-age-key-file/key
secretsAsFiles:
  - mountPath: /etc/sops-age-key-file
    name: sops-age-key-file
    secretName: sops-age-key-file
EOF

Install the SOPS Operator into our cluster

In [37]:
%%sh

helm repo add sops https://isindir.github.io/sops-secrets-operator/
helm repo update
helm upgrade --install --create-namespace sops sops/sops-secrets-operator --namespace sops --values ./kubernetes/sops-helm-values.yaml

"sops" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "postgres-operator-ui-charts" chart repository
...Successfully got an update from the "longhorn" chart repository
...Successfully got an update from the "nginx-stable" chart repository
...Successfully got an update from the "sops" chart repository
...Successfully got an update from the "postgres-operator-charts" chart repository
...Successfully got an update from the "atlassian-data-center" chart repository
...Successfully got an update from the "rancher-stable" chart repository
...Successfully got an update from the "traefik" chart repository
...Successfully got an update from the "rancher-latest" chart repository
...Successfully got an update from the "grafana" chart repository
...Successfully got an update from the "bitnami" chart repository
...Successfully got an update from the "jfrog" chart repository
Update Complet

In [39]:
%%sh
kubectl logs -n sops sops-sops-secrets-operator-54d5c5c8c8-fw85t

{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"controller-runtime.metrics","msg":"Metrics server is starting to listen","addr":":8080"}
{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"setup","msg":"SopsSecret reconciliation will be requeued after 5 minutes after decryption failures"}
{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"setup","msg":"starting manager"}
{"level":"info","ts":"2023-04-27T13:51:49Z","msg":"Starting server","path":"/metrics","kind":"metrics","addr":"[::]:8080"}
{"level":"info","ts":"2023-04-27T13:51:49Z","msg":"Starting server","kind":"health probe","addr":"[::]:8081"}
I0427 13:51:49.408175       1 leaderelection.go:248] attempting to acquire leader lease sops/ca57d051.github.com...
I0427 13:52:13.389818       1 leaderelection.go:258] successfully acquired lease sops/ca57d051.github.com
{"level":"info","ts":"2023-04-27T13:52:13Z","msg":"Starting EventSource","controller":"sopssecret","controllerGroup":"isindir.github.com","controllerKind":"Sops

Create a yaml secret with all your values and install it into the cluster

In [41]:
%%sh
cat >./kubernetes/test-secrets.yaml <<EOF
apiVersion: isindir.github.com/v1alpha3
kind: SopsSecret
metadata:
  name: example-sopssecret
spec:
  # suspend reconciliation of the sops secret object
  suspend: false
  secretTemplates:
    - name: my-secret-name-1
      labels:
        label1: value1
      annotations:
        key1: value1
      stringData:
        data-name0: data-value0
      data:
        data-name1: ZGF0YS12YWx1ZTE=
    - name: jenkins-secret
      labels:
        "jenkins.io/credentials-type": "usernamePassword"
      annotations:
        "jenkins.io/credentials-description": "credentials from Kubernetes"
      stringData:
        username: myUsername
        password: 'Pa$$word'
    - name: some-token
      stringData:
        token: Wb4ziZdELkdUf6m6KtNd7iRjjQRvSeJno5meH4NAGHFmpqJyEsekZ2WjX232s4Gj
    - name: docker-login
      type: 'kubernetes.io/dockerconfigjson'
      stringData:
        .dockerconfigjson: '{"auths":{"index.docker.io":{"username":"imyuser","password":"mypass","email":"myuser@abc.com","auth":"aW15dXNlcjpteXBhc3M="}}}'
EOF

Next step is to encrypt the secret with sops. We use `--encrypted-suffix='Templates'``. For encryption we only need the public key of the cluster in. This is saved in `.sopsrc` in our local repository

In [45]:
%%sh 
sops --encrypt --encrypted-suffix='Templates' ./kubernetes/test-secrets.yaml > ./kubernetes/test-secrets.kubesops.yaml
kubectl apply -f ./kubernetes/test-secrets.kubesops.yaml -n default

sopssecret.isindir.github.com/example-sopssecret created


If we look into the cluster, the secret has been successfully created

In [50]:
%%sh 

kubectl get secrets 
echo "------------------"
kubectl describe secrets jenkins-secret

NAME               TYPE                             DATA   AGE
docker-login       kubernetes.io/dockerconfigjson   1      4m5s
jenkins-secret     Opaque                           2      4m6s
my-secret-name-1   Opaque                           2      4m6s
some-token         Opaque                           1      4m6s
------------------
Name:         jenkins-secret
Namespace:    default
Labels:       jenkins.io/credentials-type=usernamePassword
Annotations:  jenkins.io/credentials-description: credentials from Kubernetes

Type:  Opaque

Data
====
password:  11 bytes
username:  10 bytes


The sops-secret-operator detected our change and successfully created all the secrets

In [51]:
%%sh 
kubectl logs -n sops sops-sops-secrets-operator-54d5c5c8c8-fw85t

{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"controller-runtime.metrics","msg":"Metrics server is starting to listen","addr":":8080"}
{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"setup","msg":"SopsSecret reconciliation will be requeued after 5 minutes after decryption failures"}
{"level":"info","ts":"2023-04-27T13:51:49Z","logger":"setup","msg":"starting manager"}
{"level":"info","ts":"2023-04-27T13:51:49Z","msg":"Starting server","path":"/metrics","kind":"metrics","addr":"[::]:8080"}
{"level":"info","ts":"2023-04-27T13:51:49Z","msg":"Starting server","kind":"health probe","addr":"[::]:8081"}
I0427 13:51:49.408175       1 leaderelection.go:248] attempting to acquire leader lease sops/ca57d051.github.com...
I0427 13:52:13.389818       1 leaderelection.go:258] successfully acquired lease sops/ca57d051.github.com
{"level":"info","ts":"2023-04-27T13:52:13Z","msg":"Starting EventSource","controller":"sopssecret","controllerGroup":"isindir.github.com","controllerKind":"Sops